In [1]:
import pandas as pd
import utils as keu # key_electra_utils.py 참고
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('monologg/koelectra-base-v3-discriminator')

raw_data = pd.read_csv('data/raw_book_info_list.csv',index_col=0)



,book_title,book_toc,book_intro,publisher
0,한 권으로 끝내는 메타버스 크리에이터,"['메타버스란', '왜 메타버스인가', '메타버스의 유형을 알아보자', '메타버스 ...",[],[]
1,Do it! 점프 투 파이썬: 라이브러리 예제 편,"['', '텍스트 다루기', '문자열을 줄여 표시하려면 textwrap shorte...",['이 책은 Do it 점프 투 파이썬 의 박응용 저자가 그동안 수많은 독자에게 받...,['실무에서 자주 쓰는 파이썬 라이브러리는 다 있다 필수 파이썬 라이브러리 개 엄선...


### SBert를 활용해 Sentence embedding을 만들때 모델의 Max_seq_len에 영향을 받을까?

**궁금한 점**

* koelectra 경우 Max_seq_len가 512이므로, 토큰이 512개 이상인 문장은 512개 이후에 존재하는 토큰은 버림

* 따라서 문장 하나의 토큰이 512개가 넘으면 앞의 512개의 토큰으로만 sentence embedding을 수행할 것으로 생각

* 지금 사용하는 input 데이터는 여러 문장을 하나의 문장으로 합친 다음 사용하므로 모든 문장의 토큰이 512개가 넘음

* 이런 이유로 512개 이상의 토큰을 가진 문장의 순서를 바꿈에 따라 출력 결과도 바뀔 것으로 예상.

**증명방법**

* 지금 사용하는 데이터의 [book_title, book_toc, book_intro, publisher] column은 4종류이며, 4종류의 문장을 하나로 통합해서 하나의 input data로 사용

* '파이토치 딥러닝 프로젝트 모음집' 도서의 경우 총 793개의 token으로 구분됨. 개별 column의 token 개수는 각 [5, 316, 277,195] 임.

* max_length 512이므로 column 순서로 문장을 합치면 각각 [5, 316, 191,0]개의 토큰을 활용할 것이라 생각

* 이를 대조하기 위해 원래의 column 순서를 변경해 [5, 316, 277, 195]의 토큰 개수를 [5, 277, 316, 195]로 변경하여 문장 생성, 512개까지 포함하면 [5, 277, 230, 0]개의 토큰으로 학습할 것이라 생각

* 기존 문장을 original_sentence, 새로운 문장은 new_sentence로 지칭하였음

* origin_sentence 출력과 new_sentence 출력이 다를 경우 SBert의 sentence_embedding이 모델 max_seq_len에 영향을 받는 다 생각할 수 있음.

* origin_sentence 출력과 new_sentence 출력이 같을 경우 어떻게 sentence embedding을 만드는지 추가적인 학습이 필요

**결론**
* origin_sentence 출력과 new_sentence 출력이 같음
* Sentence embedding은 모델 max_seq_len에 영향받지 않음

### Original Sentence 만들기

In [11]:
original_data = raw_data.iloc[1100] # 1100 파이토치 딥러닝 프로젝트 모음집

book_info :str =keu.Merge_Series_to_str(original_data)
book_info_trans = keu.trans_engwords_to_hanwords(book_info)

original_sentence = keu.find_han_words(' '.join(book_info_trans))

변환한 도서정보 :  파이토치 딥러닝 프로젝트 모음집


### New sentence 만들기

In [6]:
new_data = raw_data.iloc[1100].values.tolist()
new_data.append(new_data.pop(1))

book_info :str =keu.Merge_Series_to_str(new_data)
book_info_trans = keu.trans_engwords_to_hanwords(book_info)

new_sentence = keu.find_han_words(' '.join(book_info_trans))

변환한 도서정보 :  파이토치 딥러닝 프로젝트 모음집


### Original sentence vs New sentence 비교

In [13]:
print('----original----')
print(original_sentence[10:50])

print('----new----')
print(new_sentence[10:50])

----original----
['인공지능', '사례', '머신러닝', '머신', '러닝', '머신러닝이란', '머신러닝', '구분', '지도학습', '러닝', '비지도학습', '러닝', '과적합과', '모델', '학습법', '성능', '지표', '딥러닝', '딥', '러닝', '딥러닝이란', '딥러닝', '발전', '과정', '퍼셉트론', '퍼셉트론', '다층', '퍼셉트론', '레이어', '퍼셉트론', '인공신경망', '핵심', '알고리즘', '고급', '딥러닝', '기술', '뉴럴', '네트워크', '뉴럴', '네트워크']
----new----
['실제로', '우리가', '흔히', '다루는', '날것의', '데이터를', '활용한', '프로젝트', '예제집은', '찾기가', '어렵습니다', '이', '책은', '딥러닝', '프로젝트들을', '중점적으로', '다룸으로써', '어느정도', '딥러닝', '지식은', '있으나', '프로젝트로', '경험하고', '싶은', '독자들에게', '꼭', '필요한', '책이', '될', '것입니다', '이', '책은', '크게', '이론', '파트와', '실전', '파트로', '나누어져', '있습니다', '이론']


### 결과 비교
문장 순서를 바꾸더라도 결과는 같음

In [18]:
origin = keu.key_extraction(original_sentence,model)
new = keu.key_extraction(new_sentence,model)
print('----original----')
print(sorted(origin.index.tolist()))
print('----new----')
print(sorted(new.index.tolist()))

----original----
['국민청원', '다양한', '데이터', '데이터를', '딥러닝', '딥러닝을', '러닝', '분류하기', '불러오기', '이론', '있습니다', '전처리', '책으로', '책은', '크롤링', '파이토치', '파헤치기', '퍼셉트론', '프로젝트', '프로젝트로']
----new----
['국민청원', '다양한', '데이터', '데이터를', '딥러닝', '딥러닝을', '러닝', '분류하기', '불러오기', '이론', '있습니다', '전처리', '책으로', '책은', '크롤링', '파이토치', '파헤치기', '퍼셉트론', '프로젝트로', '프로젝트를']


## Electra

<br/>

### Pre-trained 된 generator를 가지고 어떻게 fake sentence를 만들 수 있을까?

**궁금한점**
* koElectra로 Domain Adaptation을 위한 설계중 어떻게 generator로 fake sentence를 만들어야 하는징 대한 문제에 직면했음

* 무엇보다 Generator는 Bert와 동일한 구조라 생각해 output이 (1,src_token_len, vocab_size)이 나와야 된다고 생각했는데, output shape이 (1,src_token_len, 256)으로 나오는게 의아했음. 

* huggingface 페이지에 작성한 활용 예시대로 활용할 경우 문제없이 작동하길래 어디서부터 잘못된 것인지 파악이 필요했음.

**증명방법**

* (1,src_token_len, 256)를 (1,src_token_len, vocab_size)로 변환하기 위해 새로운 모델을 제작

* transformers pipeline module을 뜯어보고 어떻게 output을 생성하는지 추적

**배운점**

* Encoder output을 vocab_size 크기로 변경하는 마지막 layer 또한 학습이 필요함을 경험으로 확인하였음.
    * last_hidden_state를 vocab_size 차원으로 확대 한 뒤 사용해보니 그 결과가 엉망이었음.

* Transformers 라이브러리에서 불러오는 클래스에 따라 모델의 Output이 달라짐을 이해했음.

    * Electramodel을 사용할 경우 Encoder 끝단의 output을 출력함. 출력명은 `last_hidden_state`, 크기는 `(1,src_token_len, 256)`임.
    * ElectraForMaskedLM을 사용할 경우 vocab 단어별 확률을 출력함. 출력명은 `logits` 크기는 `(1,src_token_len, vocab_size)` 임.


### 예제 불러오기

In [1]:
from transformers import pipeline, ElectraTokenizer,ElectraModel
import torch
import torch.nn as nn
from pprint import pprint
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

raw_str = '조건검색 실시간으로 [MASK]하기'
input_data = tokenizer(raw_str,return_tensors='pt')

pprint({'tokenizing' : tokenizer.tokenize(raw_str)})
pprint({'model_input' : input_data['input_ids']})



{'tokenizing': ['조건', '##검', '##색', '실시간', '##으로', '[MASK]', '하', '##기']}
{'model_input': tensor([[    2,  7080,  4166,  4703, 10460, 10749,     4,  3755,  4031,     3]])}


### ElectraModel을 활용하여 [Mask]를 예측할 경우

### 부자연스러운 결과 생성

In [16]:
generator = ElectraModel.from_pretrained("monologg/koelectra-base-v3-generator")

token_output = generator(**input_data)

print('Model_output')
print('last_hidden_state : ',token_output.last_hidden_state.shape) 


class output_generator(nn.Module) :
    '''
    last_hidden_state를 vocab_size에 맞게 차원 확장하는 모델 구현
    input_size = (1,src_token_size, 256) 
    output_size = (1,src_token_size, 35000)
    '''
    def __init__(self, output_embed : int, vocab_size : int) -> None:
        super().__init__()
        self.output_embed = output_embed
        self.vocab_embedding = nn.Linear(output_embed,vocab_size) #(256 => 35000)

    def forward(self,src) :
        return self.vocab_embedding(src) #(src_token_len, 35000)


token_logits = output_generator(256,35000) # (embed_size, vocab_size)
token_logits = token_logits(token_output.last_hidden_state)

# [mask]가 위치한 idx
mask_token_index = torch.where(input_data["input_ids"] == tokenizer.mask_token_id)[1]

# [mask]가 위치한 vocab
mask_token_logits = token_logits[0,mask_token_index-1, :]

# top 5추출
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

# 결과 추출
print('')
print("Input_data ", raw_str)
print('-'*20)
for token in top_5_tokens:
    print(f"'>>> {raw_str.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

Model_output
last_hidden_state :  torch.Size([1, 10, 256])

Input_data  조건검색 실시간으로 [MASK]하기
--------------------
'>>> 조건검색 실시간으로 ##ition하기'
'>>> 조건검색 실시간으로 가느다란하기'
'>>> 조건검색 실시간으로 〈하기'
'>>> 조건검색 실시간으로 ##잡이하기'
'>>> 조건검색 실시간으로 연예하기'


### Hugging face generator 페이지에 나온 예제 활용 하는 경우(pipeline 활용)

### 정상 작동

In [8]:
fill_mask = pipeline(
    "fill-mask",
    model="monologg/koelectra-base-v3-generator",
    tokenizer="monologg/koelectra-base-v3-generator"
)

pprint(fill_mask(raw_str))

[{'score': 0.09738744050264359,
  'sequence': '조건검색 실시간으로 확인 하기',
  'token': 6465,
  'token_str': '확인'},
 {'score': 0.07290185242891312,
  'sequence': '조건검색 실시간으로 답변 하기',
  'token': 8610,
  'token_str': '답변'},
 {'score': 0.03160829842090607,
  'sequence': '조건검색 실시간으로 진행 하기',
  'token': 6355,
  'token_str': '진행'},
 {'score': 0.0188735481351614,
  'sequence': '조건검색 실시간으로 로그인 하기',
  'token': 22864,
  'token_str': '로그인'},
 {'score': 0.017551911994814873,
  'sequence': '조건검색 실시간으로 답 하기',
  'token': 2358,
  'token_str': '답'}]


### Transformers class를 ElectraModel에서 ElectraForMaskedLM로 바꿔 사용한 경우

### 예제와 같이 정상 작동

In [18]:
from transformers import ElectraForMaskedLM

forMaskedLM = ElectraForMaskedLM.from_pretrained('monologg/koelectra-base-v3-generator')

In [23]:
raw_str = '조건검색 실시간으로 [MASK]하기'
input_data = tokenizer(raw_str,return_tensors='pt')


logits = forMaskedLM(**input_data).logits

print('Model_output')
print('logits : ',logits.shape) 


mask_token_index = torch.where(input_data["input_ids"] == tokenizer.mask_token_id)[1]



mask_token_logits = logits[0,mask_token_index, :]


top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

print('')
print("Input_data ", raw_str)
print('-'*20)

for token in top_5_tokens:
    print(f"'>>> {raw_str.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

Model_output
logits :  torch.Size([1, 10, 35000])

Input_data  조건검색 실시간으로 [MASK]하기
--------------------
'>>> 조건검색 실시간으로 확인하기'
'>>> 조건검색 실시간으로 답변하기'
'>>> 조건검색 실시간으로 진행하기'
'>>> 조건검색 실시간으로 로그인하기'
'>>> 조건검색 실시간으로 답하기'
